In [1]:
import sys
# Get path to directory with functions for dataset processing
sys.path.append('/home/pfa/Documents/Code')
import argparse
import os
import cv2
import numpy as np
import background_subtraction
import resize
import vid2jpg
# from imageio import imread
from PIL import Image
from matplotlib import pyplot as plt
from skimage import morphology
from scipy.ndimage import label
from skimage.measure import regionprops
from sort import sort_nicely

os.chdir('/home')
dir = os.getcwd()
print(dir)

# Define source and destination directory paths
pathIn = 'datasets/FCCN_Videos'
pathOut = 'pfa/Documents/GAIT-IT-RGB'

/home


In [2]:
# Lists with pathologies and subjects
pathologies = {'Diplegic': 'pat1', 'Hemiplegic': 'pat2', 'Neuropathic': 'pat3', 'Normal': 'normal', 'Parkinson': 'pat4'}
cameras = {'Camera 1': 'front_view', 'Camera 3': 'side_view'}
severity2lvl = {'s1': 'lvl1', 's2': 'lvl2', 'Normal': ''}
subs = os.listdir(pathIn)
sort_nicely(subs)
print(subs)

['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21', 's22', 's23']


In [8]:
# Iterate through pathologies to access each pathology directory
for pathology in pathologies:

    # Create folder path for each pathology in DATASETS directory
    pathology_dir = os.path.join(pathOut, pathology)
    os.mkdir(pathology_dir)

    # Create folder for each subject in each pathology directory
    for sub in subs:
        sub_dir = os.path.join(pathology_dir, sub)
        os.mkdir(sub_dir)

        # Create folder for binary silhouettes
        sub_dir_silhouettes = os.path.join(sub_dir, 'silhouettes')
        os.mkdir(sub_dir_silhouettes)

        # Create folder for each view angle in the subject silhouettes directories
        for view in ['side_view']:
            # Silhouettes
            silhouettes_view_dir = os.path.join(sub_dir_silhouettes, view)
            os.mkdir(silhouettes_view_dir)

            for severity in ['lvl1', 'lvl2']:
                
                folder_name = sub + pathologies[pathology] + severity

                if pathology == 'Normal':
                    folder_name = sub + pathologies[pathology]
                    
                # Silhouettes
                silhouettes_severity_dir = os.path.join(silhouettes_view_dir, folder_name)
                os.mkdir(silhouettes_severity_dir)

                if pathology == 'Normal':
                    break
    


In [9]:
# Iterate through subject folders
for sub in subs:
    
    sub_dir = os.path.join(pathIn, sub)

    # Get video sequences from subject
    sequences = os.listdir(sub_dir)
    sort_nicely(sequences)

    for sequence in sequences:
        
        # Source path for video
        src = os.path.join(sub_dir, sequence)

        # Destination folder for silhouettes
        for pathology in pathologies:
            if pathology in sequence:
                for camera in ['Camera 3']:
                    if camera in sequence:
                        for severity in severity2lvl:
                            if severity in sequence:
                                
                                # Final folder name: subXpatYlvlZ_Wdireaction
                                dest_folder_name = sub+pathologies[pathology]+severity2lvl[severity]

                                if pathology == 'Normal':
                                    dest_folder_name = sub + pathologies[pathology]

                                # pathology/subject/view/subXpatYlvlZ_Wdireaction
                                dest_dir = os.path.join(pathOut,pathology,sub,'silhouettes',cameras[camera],dest_folder_name)
        
                                # Decompress video into frames
                                frames = vid2jpg.extractFrames(src)

                                # silhouettes = background_subtraction.bgn_sub_MOG(frames)
                                masks, silhouettes = background_subtraction.bgn_sub_green_side_silhouettes(frames)

                                count = resize.resize_silhouettesRGB(masks, silhouettes, dest_dir)